### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [3]:
pkl_directory = '../data/company_daily_price_pkl'
pkl_directory = './'
pkl_directory = '../data/company_pkl/'

In [4]:
# investor.pkl, historical.pkl. 읽기
for key, val in code.items():
    pkl_name= '{}.pkl'.format(val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle('./' + pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime

In [5]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [81]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    print("******************", df_o_trans['before'])
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [7]:
# date column을 index로 옮김
df_inv = df_sec_investors.set_index('date')
df_his = df_sec_historical.set_index('date')
df_o = df_sec.set_index('date')

In [8]:
type(df_o.index[0]), type(df_inv.index[0]), type(df_his.index[0])

(datetime.date, datetime.date, datetime.date)

In [82]:
investor_rate = pd.DataFrame()
historical_rate = pd.DataFrame()
for date_current in df_o.index[35:37]:
    date_previous = find_previous_date(df_inv, df_his, date_current)
    date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
    df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
    df_his_comp = df_his.loc[[date_previous_1, date_previous]]
    investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
    historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)

****************** retail          -9314.0
foreigner       56530.0
institution    -46752.0
financial      -56091.0
invtrust         2008.0
pension         -1506.0
privequity       6599.0
bank              802.0
insurance        1304.0
financeetc        132.0
corporateetc      294.0
foreigneretc     -758.0
Name: before, dtype: float64
****************** open         61700.0
high         61800.0
low          61200.0
close        61500.0
close_cr        0.33
vol         561836.0
Name: before, dtype: object
****************** retail          17257.0
foreigner      -36693.0
institution     20895.0
financial       17535.0
invtrust         -891.0
pension         -5487.0
privequity       2018.0
bank              582.0
insurance        6606.0
financeetc        532.0
corporateetc    -2088.0
foreigneretc      628.0
Name: before, dtype: float64
****************** open         61700.0
high         61900.0
low          61200.0
close        61500.0
close_cr         0.0
vol         588583.0
Name: befo

ZeroDivisionError: float division by zero

In [63]:
date_current

datetime.date(2022, 8, 8)

In [76]:
df_his[885:895]

,open,high,low,close,close_cr,vol
date,,,,,,
2022-06-04,,,,,,
2022-06-05,,,,,,
2022-06-06,,,,,,
2022-06-07,66200.0,66400.0,65400.0,65500.0,-1.95,1273445.0
2022-06-08,65400.0,65700.0,65300.0,65300.0,-0.31,817379.0
2022-06-09,65100.0,65200.0,64500.0,65200.0,-0.15,1673764.0
2022-06-10,64000.0,64400.0,63800.0,63800.0,-2.15,1419245.0
2022-06-11,,,,,,
2022-06-12,,,,,,


In [34]:
investor_rate = pd.concat([investor_rate, aa], axis=0)

In [41]:
investor_rate

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
2022-06-20,8.195577,11.920678,-3.647354,7.363196,-30.591457,0.344811,23.893570,-2.813541,6.893887,-1.491670,1.854798,0.914929
2022-06-21,-0.674957,-0.407361,2.593047,0.295068,-0.938927,-0.883900,-0.821502,-0.937576,-0.955874,0.062932,-0.814356,-0.661150
2022-06-22,-0.226413,-0.166064,-0.135501,0.034853,3.990909,4.034743,-0.044910,1.406863,4.635762,-0.988289,1.836559,-1.530848
2022-06-23,-0.115765,-0.743679,-1.316192,-1.102356,-0.227930,0.236304,-4.804075,-1.851324,-0.502350,-21.944444,-0.254890,-1.760291
2022-06-24,-1.523278,3.064922,-8.766880,-20.769609,-2.370714,-0.040771,-0.727922,2.363636,-5.573790,2.116711,-2.381156,-1.818471
2022-06-27,0.124675,-0.502665,-0.416763,-0.363348,-0.698910,0.104842,2.208481,0.566145,-2.096283,-0.589787,-1.052159,-3.070039
2022-06-28,-1.992439,-1.693703,-1.819204,-1.506061,2.833841,1.641951,-0.618943,-0.727975,-2.511184,1.771784,14.149718,-2.456767
2022-06-29,0.054169,-0.882437,-0.528091,-0.883944,-1.137674,-0.172812,-2.383567,0.464107,-0.765036,-0.898204,-0.891479,-0.881290


In [47]:
historical_rate

,open,high,low,close,close_cr,vol
2022-06-20,-0.030995,-0.030744,-0.018182,-0.018062,-6.484848,0.210613
2022-06-21,0.006734,0.0,-0.021886,-0.018395,0.016575,0.152747
2022-06-22,-0.018395,-0.011686,0.001721,-0.003407,-0.815217,-0.261422
2022-06-23,0.005111,-0.001689,-0.010309,-0.015385,3.529412,-0.077913
2022-06-24,-0.022034,-0.018613,-0.013889,-0.003472,-0.772727,0.196271
2022-06-27,0.003466,0.018966,0.015845,0.017422,-5.971429,-0.16455
2022-06-28,0.018998,0.013536,0.010399,0.006849,-0.609195,-0.211953
2022-06-29,0.00339,-0.006678,0.006861,0.010204,0.5,-0.253127


In [ ]:
aa = find_ratio(df_inv_comp, date_current)
aaa = aa.iloc[[-1]]
aaa

In [ ]:
aa = find_ratio(df_inv_comp, date_current)
bbb = aa.iloc[[-1]]
bbb

In [ ]:
pd.concat([aaa, bbb], axis=0)

In [ ]:
investor_rate = pd.DataFrame()
investor_rate

In [ ]:
pd.concat([investor_rate, bbb], axis=0)

In [ ]:
aa1.iloc[-1]

In [ ]:
aa1

In [ ]:
df_o = pd.merge(df_o, aa, left_index=True, right_index=True, how='left')
df_o

In [ ]:
bb1 = aa1.iloc[-1]

In [ ]:
aa1.index[-1]

In [ ]:
df_o.loc[aa1.index[-1]] = bb1

In [ ]:
df_o

In [ ]:
bb = pd.concat([df_o, aa1], axis=0)
bb

In [ ]:
df_o.join(aa1, how='left')

In [ ]:
type(df_o.index[0])

In [ ]:
df_o.join(aa)

In [ ]:
bb = date_current.strftime('%Y-%m-%d')
bb

In [ ]:
result4 = pd.con([df1,sr1], axis=1)

In [ ]:
df_sec_investors['date'][0]

In [ ]:
df_inv_trans = df_inv_comp.transpose()
df_inv_trans.columns = ['before', 'after']
df_inv_trans['perc'] = (df_inv_trans['after'] - df_inv_trans['before']) / df_inv_trans['before']
df_inv = df_inv_trans.transpose()

In [ ]:
df_inv_trans['perc'] = (df_inv_trans['after'] - df_inv_trans['before']) / df_inv_trans['before']

In [ ]:
df_inv = df_inv_trans.transpose()

In [ ]:
def calc_percent(x, y) :
    return (y - x) / x

In [ ]:
del df_nn
df_nn = df_inv_trans

In [ ]:
df_nn ['perc'] = (df_nn['after'] - df_nn['before']) / df_nn['before']

In [ ]:
df_nn

In [ ]:
df_inv_trans.columns = ['before', 'after']

In [ ]:
df_inv_comp.apply()

In [ ]:
def calc_increment_percent(df_inv, df_his, date_current):
    date_previous = find_previous_date(df_inv, df_his, date_current)
    df_inv_comp = df_inv.loc[[date_previous, date_current]]
    df_his_comp = df_his.loc[[date_previous, date_current]]
    return inc

In [ ]:
date1 = datetime.datetime.strptime(df_sec.loc[0, 'date'], '%Y-%m-%d')
# date1 = datetime.datetime.strptime('2022-06-16', '%Y-%m-%d')
aa = find_previous_date(df_inv, df_his, date1)

In [ ]:
aa

In [ ]:
date1 = datetime.datetime.strptime('2022-06-17', '%Y-%m-%d')
bb = find_previous_date(df_inv, df_his, date1)
bb